In [ ]:
# Import packages
import os
from zipfile import ZipFile
import boto3
import pprint

In [ ]:
# Put credentials file in .aws directory; check if file is found
# https://docs.aws.amazon.com/credref/latest/refdocs/file-location.html
# %USERPROFILE%\.aws\credentials

os.listdir(os.environ['USERPROFILE'] + '\.aws\\')

In [ ]:
# Create S3 bucket
s3 = boto3.client('s3')
response = s3.create_bucket(
    Bucket='weisman-sqs-test'
)

pprint.pprint(response)

In [ ]:
sqs.delete_queue(QueueUrl='https://queue.amazonaws.com/475434563362/weisman-sqs-test')

In [ ]:
# Instantiate SQS client
sqs = boto3.client('sqs', 'us-east-1')

# Create a queue
response = sqs.create_queue(
    QueueName='weisman-sqs-test',
    Attributes={
        'DelaySeconds': '0',
        'ReceiveMessageWaitTimeSeconds': '0',
        'VisibilityTimeout':'1'
    }
)

pprint.pprint(response)

In [ ]:
# Zip the .py file that contains the lambda function
ZipFile('reference/deploy3.zip', mode='w').write('reference/lambda_function3.py')
ZipFile('reference/deploy4.zip', mode='w').write('reference/lambda_function4.py')

In [ ]:
# Instantiate Lamba client
awsLambda = boto3.client('lambda', 'us-east-1')

# Create a Lambda that writes message from queue to S3
response1 = awsLambda.create_function(
    FunctionName='weisman-sqs-process-test',
    Runtime='python3.8',
    Role='arn:aws:iam::475434563362:role/service-role/weisman-test-role-03ak8xfv', # copied from IAM
    Handler='string',
    Code={
      'ZipFile': open('reference/deploy3.zip', 'rb').read()
    },
    Description='Lambda function to process messages from queue',
    Timeout=1,
    Publish=False,)

# Add SQS trigger to Lambda cunction
response2 = awsLambda.create_event_source_mapping(
    BatchSize=1,
    EventSourceArn='arn:aws:sqs:us-east-1:475434563362:weisman-sqs-test',
    FunctionName='weisman-sqs-process-test',
    MaximumBatchingWindowInSeconds=123
)

pprint.pprint(response1)
pprint.pprint(response2)

In [ ]:
# Create a Lambda function that sends messages to SQS
response = awsLambda.create_function(
    FunctionName='weisman-sqs-generate-test',
    Runtime='python3.8',
    Role='arn:aws:iam::475434563362:role/service-role/weisman-test-role-03ak8xfv', # copied from IAM
    Handler='string',
    Code={
      'ZipFile': open('reference/deploy4.zip', 'rb').read()
    },
    Description='Lambda function to generate messages to send to queue',
    Timeout=123,
    Publish=False)

pprint.pprint(response)

In [ ]:
# Invoke Lambda function to generate a message
awsLambda.invoke(
    FunctionName='weisman-sqs-generate-test',
    InvocationType='Event')

In [ ]:
# Check S3 bucket to see if message successfully got processed
response = s3.list_objects(Bucket='weisman-sqs-test')
pprint.pprint(response)